In [1]:
from geoband.API import *
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import *
#pd.set_option('display.max_rows', None)

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [27]:
# 작업한 파일 로컬로 다운로드 함수
def csv_download_link(df, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    df.to_csv(csv_file_name, index=False)
    from IPython.display import FileLink
    display(FileLink(csv_file_name))
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [2]:
police = gpd.read_file('./data/1.5개시도_경찰서_관할경계.geojson')
securelight = pd.read_csv('./data/9.5개시도_보안등현황.csv')

In [3]:
securelight.rename(columns={'\nsecuritylight_cnt' : 'securitylight_cnt'}, inplace=True)

In [11]:
df = pd.read_csv('./base_df.csv')
plist = df['jur_stn'].unique()
yearseries = df['year'].unique()
yearseries = yearseries.tolist()
# yearseries.append('2016(이전)')

In [13]:
plist

array(['서울중부경찰서', '서울종로경찰서', '서울남대문경찰서', '서울서대문경찰서', '서울혜화경찰서', '서울용산경찰서',
       '서울성북경찰서', '서울동대문경찰서', '서울마포경찰서', '서울영등포경찰서', '서울성동경찰서', '서울동작경찰서',
       '서울광진경찰서', '서울서부경찰서', '서울강북경찰서', '서울금천경찰서', '서울중랑경찰서', '서울강남경찰서',
       '서울관악경찰서', '서울강서경찰서', '서울강동경찰서', '서울종암경찰서', '서울구로경찰서', '서울서초경찰서',
       '서울양천경찰서', '서울송파경찰서', '서울노원경찰서', '서울방배경찰서', '서울은평경찰서', '서울도봉경찰서',
       '서울수서경찰서', '수원중부경찰서', '수원남부경찰서', '수원서부경찰서', '세종경찰서', '창원중부경찰서',
       '창원서부경찰서', '마산중부경찰서', '마산동부경찰서', '진주경찰서', '진해경찰서'], dtype=object)

In [14]:
yearseries

['2016(이전)',
 '2017(상반기)',
 '2017(하반기)',
 '2018(상반기)',
 '2018(하반기)',
 '2019(상반기)',
 '2019(하반기)',
 '2020(상반기)',
 '2020(하반기)']

In [15]:
securelight['lon'] = securelight['lon'].astype(float)
securelight['lat'] = securelight['lat'].astype(float)
securelight['geometry'] = securelight.apply(lambda row:Point([row['lon'],row['lat']]), axis=1)
securelight = gpd.GeoDataFrame(securelight,geometry='geometry',crs="EPSG:4326")

In [16]:
securelight = gpd.sjoin(securelight,police,how='left',op='within')
securelight.drop(['index_right','PNAME'],axis=1,inplace=True)

In [17]:
securelight.rename(columns = {"NAME":"jur_stn"}, inplace=True)

In [18]:
securelight

,year,address,securitylight_cnt,lon,lat,geometry,jur_stn
0,NaN,세종특별자치시 금남면 감성리 64-2,1,127.287690,36.443467,POINT (127.28769 36.44347),세종경찰서
1,NaN,세종특별자치시 금남면 감성리 267,1,127.288812,36.444181,POINT (127.28881 36.44418),세종경찰서
2,NaN,세종특별자치시 금남면 감성리 40-1,1,127.289575,36.444711,POINT (127.28957 36.44471),세종경찰서
3,NaN,세종특별자치시 금남면 감성리 26,1,127.290071,36.444455,POINT (127.29007 36.44445),세종경찰서
4,NaN,세종특별자치시 금남면 감성리 267,1,127.290002,36.444188,POINT (127.29000 36.44419),세종경찰서
...,...,...,...,...,...,...,...
229400,NaN,서울특별시 중랑구 상봉동 19-2,1,127.092434,37.602786,POINT (127.09243 37.60279),서울중랑경찰서
229401,NaN,서울특별시 중랑구 상봉동 19-44,1,127.092721,37.602497,POINT (127.09272 37.60250),서울중랑경찰서
229402,NaN,서울특별시 중랑구 상봉동 19-25,1,127.092258,37.602374,POINT (127.09226 37.60237),서울중랑경찰서
229403,NaN,서울특별시 중랑구 상봉동 495-4,1,127.094617,37.602357,POINT (127.09462 37.60236),서울중랑경찰서


In [19]:
securelight.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 229405 entries, 0 to 229404
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   year               87225 non-null   float64 
 1   address            218426 non-null  object  
 2   securitylight_cnt  229405 non-null  int64   
 3   lon                229405 non-null  float64 
 4   lat                229405 non-null  float64 
 5   geometry           229405 non-null  geometry
 6   jur_stn            229369 non-null  object  
dtypes: float64(3), geometry(1), int64(1), object(2)
memory usage: 14.0+ MB


In [20]:
#보안등
securelight.dropna(subset=['jur_stn'],inplace=True)
securelight_year = securelight['year'].fillna(1900.0)
securelight.drop(['year'],axis=1,inplace=True)
securelight['year'] = securelight_year
securelight = securelight.astype({'year': int})

In [22]:
securelight

,address,securitylight_cnt,lon,lat,geometry,jur_stn,year
0,세종특별자치시 금남면 감성리 64-2,1,127.287690,36.443467,POINT (127.28769 36.44347),세종경찰서,1900
1,세종특별자치시 금남면 감성리 267,1,127.288812,36.444181,POINT (127.28881 36.44418),세종경찰서,1900
2,세종특별자치시 금남면 감성리 40-1,1,127.289575,36.444711,POINT (127.28957 36.44471),세종경찰서,1900
3,세종특별자치시 금남면 감성리 26,1,127.290071,36.444455,POINT (127.29007 36.44445),세종경찰서,1900
4,세종특별자치시 금남면 감성리 267,1,127.290002,36.444188,POINT (127.29000 36.44419),세종경찰서,1900
...,...,...,...,...,...,...,...
229400,서울특별시 중랑구 상봉동 19-2,1,127.092434,37.602786,POINT (127.09243 37.60279),서울중랑경찰서,1900
229401,서울특별시 중랑구 상봉동 19-44,1,127.092721,37.602497,POINT (127.09272 37.60250),서울중랑경찰서,1900
229402,서울특별시 중랑구 상봉동 19-25,1,127.092258,37.602374,POINT (127.09226 37.60237),서울중랑경찰서,1900
229403,서울특별시 중랑구 상봉동 495-4,1,127.094617,37.602357,POINT (127.09462 37.60236),서울중랑경찰서,1900


In [21]:
securelight.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 229369 entries, 0 to 229404
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   address            218390 non-null  object  
 1   securitylight_cnt  229369 non-null  int64   
 2   lon                229369 non-null  float64 
 3   lat                229369 non-null  float64 
 4   geometry           229369 non-null  geometry
 5   jur_stn            229369 non-null  object  
 6   year               229369 non-null  int64   
dtypes: float64(2), geometry(1), int64(2), object(2)
memory usage: 14.0+ MB


In [23]:
ysh = []

for idx, row in securelight.iterrows():
    if row['year'] == 2017:
        ym = '2017(상반기)'
    elif row['year'] == 2018:
        ym = '2018(상반기)'
    elif row['year'] == 2019:
        ym = '2019(상반기)'
        
    ## 기존코드에서 2020 누락되어 있어서 추가함
    elif row['year'] == 2020:
        ym = '2020(상반기)'
        
    elif row['year'] < 2017:
        ym = '2016(이전)'
    else:
        ym = row['year']
    #print(ym)
    ysh.append(ym)

securelight.drop(['year','address','lat','lon','geometry'],axis=1,inplace=True)    
securelight['year'] = ysh

# securelightjur = securelight.groupby(['jur_stn','year'])[['securitylight_cnt']].sum()

In [24]:
securelight

,securitylight_cnt,jur_stn,year
0,1,세종경찰서,2016(이전)
1,1,세종경찰서,2016(이전)
2,1,세종경찰서,2016(이전)
3,1,세종경찰서,2016(이전)
4,1,세종경찰서,2016(이전)
...,...,...,...
229400,1,서울중랑경찰서,2016(이전)
229401,1,서울중랑경찰서,2016(이전)
229402,1,서울중랑경찰서,2016(이전)
229403,1,서울중랑경찰서,2016(이전)


In [31]:
# 21년 작업이 어떻게 될지 모르니 일단 저장
securelight.to_csv('./9번.보안등_processed.csv', index=False)

In [33]:
securelight = pd.read_csv('./9번.보안등_processed.csv')
securelight

,securitylight_cnt,jur_stn,year
0,1,세종경찰서,2016(이전)
1,1,세종경찰서,2016(이전)
2,1,세종경찰서,2016(이전)
3,1,세종경찰서,2016(이전)
4,1,세종경찰서,2016(이전)
...,...,...,...
229364,1,서울중랑경찰서,2016(이전)
229365,1,서울중랑경찰서,2016(이전)
229366,1,서울중랑경찰서,2016(이전)
229367,1,서울중랑경찰서,2016(이전)


In [28]:
# 데이터 백업을 위한 로컬 저장소 활용
csv_download_link(securelight,'9번.보안등_processed.csv')

/opt/app-root/src/9번.보안등_processed.csv

Press enter to delete the file after you have downloaded it.


## count

In [39]:
securelight['year'].unique()

array(['2016(이전)', '2019(상반기)', '2017(상반기)', '2018(상반기)', '2020(상반기)',
       '2021'], dtype=object)

In [40]:
securelight = securelight[securelight['year']!='2021']
securelight

,securitylight_cnt,jur_stn,year
0,1,세종경찰서,2016(이전)
1,1,세종경찰서,2016(이전)
2,1,세종경찰서,2016(이전)
3,1,세종경찰서,2016(이전)
4,1,세종경찰서,2016(이전)
...,...,...,...
229364,1,서울중랑경찰서,2016(이전)
229365,1,서울중랑경찰서,2016(이전)
229366,1,서울중랑경찰서,2016(이전)
229367,1,서울중랑경찰서,2016(이전)


In [42]:
securelightjur = securelight.groupby(['jur_stn','year'])[['securitylight_cnt']].sum()
securelightjur.reset_index(inplace=True)
securelightjur

,jur_stn,year,securitylight_cnt
0,마산동부경찰서,2016(이전),5647
1,마산중부경찰서,2016(이전),6821
2,서울강남경찰서,2016(이전),5029
3,서울강동경찰서,2016(이전),7500
4,서울강북경찰서,2016(이전),8431
...,...,...,...
73,진주경찰서,2016(이전),6147
74,진주경찰서,2019(상반기),6
75,진해경찰서,2016(이전),3218
76,창원서부경찰서,2016(이전),6078


In [43]:
base_df = pd.read_csv('./base_df.csv')
base_df

,jur_stn,year
0,서울중부경찰서,2016(이전)
1,서울종로경찰서,2016(이전)
2,서울남대문경찰서,2016(이전)
3,서울서대문경찰서,2016(이전)
4,서울혜화경찰서,2016(이전)
...,...,...
364,창원서부경찰서,2020(하반기)
365,마산중부경찰서,2020(하반기)
366,마산동부경찰서,2020(하반기)
367,진주경찰서,2020(하반기)


In [46]:
securelightjur = pd.merge(base_df,securelightjur, how= 'outer', on=('jur_stn','year'))
securelightjur.fillna(0.0,inplace=True)
securelightjur

,jur_stn,year,securitylight_cnt
0,서울중부경찰서,2016(이전),3430.0
1,서울종로경찰서,2016(이전),5259.0
2,서울남대문경찰서,2016(이전),857.0
3,서울서대문경찰서,2016(이전),6938.0
4,서울혜화경찰서,2016(이전),4256.0
...,...,...,...
364,창원서부경찰서,2020(하반기),0.0
365,마산중부경찰서,2020(하반기),0.0
366,마산동부경찰서,2020(하반기),0.0
367,진주경찰서,2020(하반기),0.0


In [47]:
securelightjur.to_csv('./9번.보안등_total.csv')

In [48]:
# 데이터 백업을 위한 로컬 저장소 활용
csv_download_link(securelightjur,'9번.보안등_total.csv')

/opt/app-root/src/9번.보안등_total.csv

Press enter to delete the file after you have downloaded it.


In [16]:
for i in plist:
    for j in yearseries:
        if not securelightjur.index.isin([(i, j)]).any():
            securelightjur.loc[(i, j), "securitylight_cnt"] = int(0)

In [17]:
pd.set_option('display.max_rows', None)

In [18]:
securelightjur

,,securitylight_cnt
jur_stn,year,
마산동부경찰서,2016(이전),5647.0
마산중부경찰서,2016(이전),6821.0
서울강남경찰서,2016(이전),5029.0
서울강동경찰서,2016(이전),7500.0
서울강북경찰서,2016(이전),8431.0
서울강서경찰서,2016(이전),8573.0
서울관악경찰서,2016(이전),11882.0
서울광진경찰서,2016(이전),9132.0
서울구로경찰서,2020,925.0


In [19]:
securelightjur=securelightjur.reset_index('jur_stn')
securelightjur=securelightjur.reset_index('year')
securelightjur.sort_values(['jur_stn', 'year'], ascending=[True, True],inplace=True)

In [20]:
securelightjur_final = pd.DataFrame(
                  index=[securelightjur['jur_stn'], securelightjur['year']],
                  columns=['securitylight'])
securelightjur_final['securitylight'] = securelightjur['securitylight_cnt'].tolist()
securelightjur_final['securitylight'] = securelightjur_final['securitylight'].astype(int)

In [21]:
securelight=securelightjur_final.reset_index('year')
securelight=securelight.reset_index('jur_stn')

In [22]:
securelight

,jur_stn,year,securitylight
0,마산동부경찰서,2016(이전),5647
1,마산동부경찰서,2017(상반기),0
2,마산동부경찰서,2017(하반기),0
3,마산동부경찰서,2018(상반기),0
4,마산동부경찰서,2018(하반기),0
5,마산동부경찰서,2019(상반기),0
6,마산동부경찰서,2019(하반기),0
7,마산동부경찰서,2020(상반기),0
8,마산동부경찰서,2020(하반기),0
9,마산중부경찰서,2016(이전),6821


In [24]:
securelight.to_csv("9.securelight.csv", index=False)